# Error analysis for Context-Aware ALD Models 

This notebook contrains the error and qualitative analysis for different models trained on CAD for Abusive Language Detection.

## Set up 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
import json
import seaborn as sns
import os 
import csv

## Read output test file 

In [2]:
file_path = "../../modernbert-class_cad_eval_test_outputs.jsonl"


In [3]:
# Open the jsonl file and read it line by line
def get_error_stats(file_path):
    true_predictions, tp, tn, false_predictions, fp, fn = 0, 0, 0, 0, 0, 0
    model_pred_1, model_pred_0 = 0, 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Parse the JSON object from each line
            json_obj = json.loads(line.strip())
            if int(json_obj['pred_label']) == int(json_obj['y']):
                true_predictions += 1
                if int(json_obj['pred_label']) == 1:
                    tp += 1
                else:
                    tn += 1
            else:
                false_predictions += 1
                if int(json_obj['pred_label']) == 1:
                    fp += 1
                else:
                    fn += 1
            
            if int(json_obj['pred_label']) == 1:
                model_pred_1 += 1
            else:
                model_pred_0 += 1

    return {
        "true_predictions": true_predictions,
        "true_positives": tp,
        "true_negatives": tn,
        "false_predictions": false_predictions,
        "false_positives": fp,
        "false_negatives": fn,
        "model_pred_1": model_pred_1,
        "model_pred_0": model_pred_0
    }



In [4]:
models = ["bert-class", "modernbert-class", "bert-concat", "bertwithneighconcat", "gat-test"]
stats_list = []

for model in models:
    file_path = os.path.join("../..", model + "_cad_eval_test_outputs.jsonl")
    print("Model: ", model)
    stats = get_error_stats(file_path)
    stats["model"] = model  # Add model name to the stats
    stats_list.append(stats)

# Write stats to a CSV file
output_file = "model_error_stats.csv"
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=[
        "model", "true_predictions", "true_positives", "true_negatives",
        "false_predictions", "false_positives", "false_negatives",
        "model_pred_1", "model_pred_0"
    ])
    writer.writeheader()
    writer.writerows(stats_list)

print(f"Stats written to {output_file}")

Model:  bert-class
Model:  modernbert-class
Model:  bert-concat
Model:  bertwithneighconcat
Model:  gat-test
Stats written to model_error_stats.csv


Get the correspondance between comment/post ids and graph files.

In [5]:
# output content of 100 first graph objetcs into the sample-reanno folder in data folder
graph_input_dir = "../../data/processed_graphs/processed/"
index_file = "../../data/cad-test-idx-many.txt"
output_file = "eval_set_output.csv"
output_dic = {}

def extract_info_from_graph(index_file, output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['filename', 'id', 'reddit_url', 'label', 'anno_ctx', 'anno_tgt', 'anno_tgt_cat', 'body', 'index_in_conv', 'conv_len']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        with open(index_file, "r") as file:
            indices = file.readlines()
            indices = [int(index.strip()) for index in indices]
            for index in indices:
                input_file = f"{graph_input_dir}graph-{index}.pt"
                try:
                    # Load the.pt file
                    graph = torch.load(input_file)
                    true_index = [i for i in range(len(graph.y_mask)) if graph.y_mask[i] == True]
                    assert len(true_index) == 1
                    true_index = true_index[0]

                    comment = graph.x_text[true_index]
                    x, a2, a3, label = comment
                    my_id = x.get('id', '')
                    permalink = x.get('permalink', '')
                    reddit_url = 'https://www.reddit.com' + permalink
                    
                    label = x.get('label', '')
                    anno_ctx = x.get('anno_ctx', '')
                    anno_tgt = x.get('anno_tgt', '')
                    anno_tgt_cat = x.get('anno_tgt_cat', '')
                    body = x.get('body', '')
                    
                    # Write to CSV
                    writer.writerow({
                        'filename': input_file,
                        'id': my_id,
                        'reddit_url': reddit_url,
                        'label': label,
                        'anno_ctx': anno_ctx,
                        'anno_tgt': anno_tgt,
                        'anno_tgt_cat': anno_tgt_cat,
                        'body': body,
                        'index_in_conv': true_index,
                        'conv_len': len(graph.y_mask)
                    })

                    #target_comment = graph.x_text[true_index]
                    #print(target_comment)


                except FileNotFoundError:
                    print(f"File {input_file} not found.")


extract_info_from_graph(index_file, output_file)


/Users/celianouri/anaconda3/envs/hatedisc/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Check model errors

#### bot-gat-dir-3l-cad-512-7_3625338_eval_outputs

In [8]:
%pwd

'/Users/celianouri/Stage24/HatefulDiscussionsModeling/model_hateful_comments/notebooks'

In [27]:
import json
import csv

# Input JSONL file
input_file = "/Users/celianouri/Stage24/HatefulDiscussionsModeling/model_hateful_comments/bot-gat-dir-3l-cad-512-42(best)_eval_outputs.jsonl"
# Output CSV file
output_file = "/Users/celianouri/Stage24/HatefulDiscussionsModeling/model_hateful_comments/bot-gat-dir-3l-cad-512-42(best)_eval_outputs.csv"

# Read the JSONL file and extract the data
data = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        obj = json.loads(line.strip())
        obj['tp'], obj['tn'], obj['fp'], obj['fn'] = 0, 0, 0, 0
        if int(obj['pred_label']) == int(obj['y']):
            if int(obj['pred_label']) == 1:
                obj['tp'] = 1
            else:
                obj['tn'] = 1
        else:
            if int(obj['pred_label']) == 1:
                obj['fp'] = 1
            else:
                obj['fn'] = 1
        data.append(obj)

# Flatten and normalize arrays or complex fields into strings
def flatten_value(value):
    if isinstance(value, list):
        return ";".join(map(str, value))  # Join list elements with semicolons
    return value

# Get all unique keys in the JSON objects (assuming consistent fields)
if data:
    headers = data[0].keys()
else:
    raise ValueError("The JSONL file is empty or malformed.")

# Write to CSV
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for item in data:
        flattened_item = {key: flatten_value(value) for key, value in item.items()}
        writer.writerow(flattened_item)

print(f"Conversion complete! Data saved to {output_file}")


Conversion complete! Data saved to /Users/celianouri/Stage24/HatefulDiscussionsModeling/model_hateful_comments/bot-gat-dir-3l-cad-512-42(best)_eval_outputs.csv


In [28]:
import pandas as pd

# Create a DataFrame
bert_results = pd.read_csv("../bertclass-cad-512-123_3625347_eval_outputs.csv")
gat3l_results = pd.read_csv("../bot-gat-dir-3l-cad-512-7_3625338_eval_outputs.csv")
gat2l_results = pd.read_csv("../bot-gat-dir-2l-cad-512-7_3624560_eval_outputs.csv")
best_bert_results = pd.read_csv("../bertclass-cad-512-42(best)_eval_outputs.csv")
best_graph_results = pd.read_csv("../bot-gat-dir-3l-cad-512-42(best)_eval_outputs.csv")



## How does this model perform on context specific samples?

In [29]:
best_bert_results_ctx = best_bert_results[best_bert_results['anno_ctx'] == 'PreviousContent']
best_graph_results_ctx = best_graph_results[best_graph_results['anno_ctx'] == 'PreviousContent']

In [30]:
best_graph_results_ctx.head()

,id,reddit_url,index,text,anno_ctx,anno_tgt,anno_tgt_cat,label,y,pred_label,y_pred,texts,conv_indices_to_keep,tp,tn,fp,fn
3,etnmw0w,https://www.reddit.com/r/TumblrInAction/commen...,46,"Feminist, Feminism, *Funny, LOL*",PreviousContent,feminists,political affiliation,AffiliationDirectedAbuse,1.0,1,"[-1.8421907424926758, 1.8497545719146729]",;Normalise masturbation? What does this ever m...,0;1;2;9;12;13;23;24;37;38;46,1,0,0,0
17,eteyggm,https://www.reddit.com/r/Drama/comments/cbchrd...,74,Gypsies of peace strike again.,PreviousContent,romani people,ethnicity,IdentityDirectedAbuse,1.0,1,"[-2.5046000480651855, 2.482245445251465]",;Trumps concentration camps are open for busin...,0;3;32;45;46;60;61;67;68;74,1,0,0,0
18,eio7kct,https://www.reddit.com/r/4chan/comments/b1ta8a...,7,"The first guy was trying to run away, can clea...",PreviousContent,brenton tarrant,hateful entities,IdentityDirectedAbuse,1.0,1,"[-1.2868059873580933, 1.2466493844985962]",;>Shoots man bravely attempting to disarm him\...,0;3;7,1,0,0,0
23,efxvf7g,https://www.reddit.com/r/4chan/comments/anszg4...,136,I don't need a fucking weebs opinion,PreviousContent,NaN,NaN,PersonDirectedAbuse,1.0,1,"[-3.4599242210388184, 3.4274837970733643]",;Brazil? Gonna need an explanation for that on...,0;1;2;67;72;78;80;131;135;136;139;140;142;143,1,0,0,0
26,eofd2ju,https://www.reddit.com/r/conspiracy/comments/b...,2,Our responsibilities lol. The arrogance and en...,PreviousContent,people from america,nationality/origin,IdentityDirectedAbuse,1.0,1,"[-2.6680080890655518, 2.721731185913086]",;###[Meta] Sticky Comment\n\n[Rule 2](https://...,0;1;2,1,0,0,0


In [14]:
best_bert_results_ctx.head()

,id,reddit_url,index,text,anno_ctx,anno_tgt,anno_tgt_cat,label,y,pred_label,y_pred,x,masked_index,tp,tn,fp,fn
3,etnmw0w,https://www.reddit.com/r/TumblrInAction/commen...,46,"Feminist, Feminism, *Funny, LOL*",PreviousContent,feminists,political affiliation,AffiliationDirectedAbuse,1.0,1,"[-1.1428617238998413, 1.2060739994049072]","{'id': 'etnmw0w', 'name': 't1_etnmw0w', 'autho...",46,1,0,0,0
17,eteyggm,https://www.reddit.com/r/Drama/comments/cbchrd...,74,Gypsies of peace strike again.,PreviousContent,romani people,ethnicity,IdentityDirectedAbuse,1.0,1,"[-1.4758193492889404, 1.5208029747009277]","{'id': 'eteyggm', 'name': 't1_eteyggm', 'autho...",74,1,0,0,0
18,eio7kct,https://www.reddit.com/r/4chan/comments/b1ta8a...,7,"The first guy was trying to run away, can clea...",PreviousContent,brenton tarrant,hateful entities,IdentityDirectedAbuse,1.0,1,"[-0.03366149216890335, 0.3167352080345154]","{'id': 'eio7kct', 'name': 't1_eio7kct', 'autho...",7,1,0,0,0
23,efxvf7g,https://www.reddit.com/r/4chan/comments/anszg4...,136,I don't need a fucking weebs opinion,PreviousContent,NaN,NaN,PersonDirectedAbuse,1.0,1,"[-1.526110291481018, 1.7947399616241455]","{'id': 'efxvf7g', 'name': 't1_efxvf7g', 'autho...",136,1,0,0,0
26,eofd2ju,https://www.reddit.com/r/conspiracy/comments/b...,2,Our responsibilities lol. The arrogance and en...,PreviousContent,people from america,nationality/origin,IdentityDirectedAbuse,1.0,1,"[-1.7145277261734009, 1.851286768913269]","{'id': 'eofd2ju', 'name': 't1_eofd2ju', 'autho...",2,1,0,0,0


In [32]:
summary_ctx = best_bert_results_ctx[['tp', 'tn', 'fp', 'fn']].sum()
print("BERT model with best performance on context 'PreviousContent':")
print(summary_ctx)

best_bert_results_currctx = best_bert_results[best_bert_results['anno_ctx'] == 'CurrentContent']
summary_currctx = best_bert_results_currctx[['tp', 'tn', 'fp', 'fn']].sum()
print("BERT model with best performance on context 'CurrentContext':")
print(summary_currctx)

print(f"Best BERT model is wrong on {summary_ctx['fn']*100/(summary_ctx['fn'] + summary_ctx['tp'])} % of Previous Context cases.")
print(f"Best BERT model is wrong on {summary_currctx['fn']*100/(summary_currctx['fn'] + summary_currctx['tp'])} % of Current Context cases.")


BERT model with best performance on context 'PreviousContent':
tp    80
tn     2
fp     1
fn    30
dtype: int64
BERT model with best performance on context 'CurrentContext':
tp    201
tn      0
fp      2
fn     37
dtype: int64
Best BERT model is wrong on 27.272727272727273 % of Previous Context cases.
Best BERT model is wrong on 15.546218487394958 % of Current Context cases.


In [33]:
summary_ctx = best_graph_results_ctx[['tp', 'tn', 'fp', 'fn']].sum()
print("Graph model with best performance on context 'PreviousContent':")
print(summary_ctx)

best_graph_results_currctx = best_graph_results[best_graph_results['anno_ctx'] == 'CurrentContent']
summary_currctx = best_graph_results_currctx[['tp', 'tn', 'fp', 'fn']].sum()
print("Graph model with best performance on context 'CurrentContext':")
print(summary_currctx)

print(f"Best Graph model is wrong on {summary_ctx['fn']*100/(summary_ctx['fn'] + summary_ctx['tp'])} % of Previous Context cases.")
print(f"Best Graph model is wrong on {summary_currctx['fn']*100/(summary_currctx['fn'] + summary_currctx['tp'])} % of Current Context cases.")

Graph model with best performance on context 'PreviousContent':
tp    86
tn     2
fp     1
fn    24
dtype: int64
Graph model with best performance on context 'CurrentContext':
tp    206
tn      0
fp      2
fn     32
dtype: int64
Best Graph model is wrong on 21.818181818181817 % of Previous Context cases.
Best Graph model is wrong on 13.445378151260504 % of Current Context cases.


### Model error analysis

In [34]:
bert_errors = bert_results[(bert_results["fp"] == 1) | (bert_results["fn"] == 1)]
gat3l_errors = gat3l_results[(gat3l_results["fp"] == 1) | (gat3l_results["fn"] == 1)]
gat2l_errors = gat2l_results[(gat2l_results["fp"] == 1) | (gat2l_results["fn"] == 1)]
best_bert_errors = best_bert_results[(best_bert_results["fp"] == 1) | (best_bert_results["fn"] == 1)]
best_graph_errors = best_graph_results[(best_graph_results["fp"] == 1) | (best_graph_results["fn"] == 1)]


In [35]:
best_bert_errors["model"] = "bert no context"
best_graph_errors["model"] = "Best graph model (3l)"

/var/folders/r7/0255tsrx2dscctsn_23l9c300000gn/T/ipykernel_36125/1321849656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_bert_errors["model"] = "bert no context"
/var/folders/r7/0255tsrx2dscctsn_23l9c300000gn/T/ipykernel_36125/1321849656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_graph_errors["model"] = "Best graph model (3l)"


In [38]:
print("Best BERT model errors: false positives and false negatives")
summary_err = best_bert_errors[['fp', 'fn']].sum()
print(summary_err)

print("Best Graph model errors: false positives and false negatives")
summary_err = best_graph_errors[['fp', 'fn']].sum()
print(summary_err)


Best BERT model errors: false positives and false negatives
fp    112
fn     68
dtype: int64
Best Graph model errors: false positives and false negatives
fp    114
fn     57
dtype: int64


In [40]:

# Merge with an indicator column
merged_df = best_graph_errors.merge(best_bert_errors, on='id', how='outer', indicator=True)

# Compute statistics
unique_in_graph = (merged_df['_merge'] == 'left_only').sum()
unique_in_bert = (merged_df['_merge'] == 'right_only').sum()
unique_in_both = (merged_df['_merge'] == 'both').sum()
total_in_graph = gat3l_errors['id'].nunique()
total_in_bert = bert_errors['id'].nunique()

# Print results
print("Number of unique IDs in GAT 3l errors:", unique_in_graph)
print("Number of unique IDs in BERT no context errors:", unique_in_bert)
print("Number of unique IDs that are errors for both models:", unique_in_both)
print("Number of errors for GAT 3l:", total_in_graph)
print("Number of errors for BERT no context:", total_in_bert)

graph_only_err = merged_df[(merged_df['_merge'] == 'left_only')]
bert_only_err = merged_df[(merged_df['_merge'] == 'right_only')]


# Save rows_only_in_df1 as a CSV file
#unique_in_gat3l.to_csv('errors_for_gat3l_but_not_bert.csv', index=False)

# Save rows_only_in_df2 as a CSV file
#unique_in_bert.to_csv('errors_for_bert_but_not_gat3l.csv', index=False)

Number of unique IDs in GAT 3l errors: 31
Number of unique IDs in BERT no context errors: 40
Number of unique IDs that are errors for both models: 140
Number of errors for GAT 3l: 179
Number of errors for BERT no context: 186


In [42]:
# Errors that were solved by the Graph model over BERT
solved_by_graph = merged_df[(merged_df['_merge'] == 'right_only')]

print("Number of errors solved by the Graph model over BERT:", len(solved_by_graph))

Number of errors solved by the Graph model over BERT: 40


In [43]:
# Errors that remain errors for both models
errors_in_both = merged_df[(merged_df['_merge'] == 'both')]

print("Number of errors that remain errors for both models:", len(errors_in_both))

Number of errors that remain errors for both models: 140


In [45]:
# Save rows_only_in_df1 as a CSV file
solved_by_graph.to_csv('solved_by_graph(_over_bert_noctxt).csv', index=False)

# Save rows_only_in_df2 as a CSV file
#unique_in_bert.to_csv('errors_for_bert_but_not_gat3l.csv', index=False)

#### Older code for old bert and old gat 3l models


In [8]:
gat3l_errors["model"] = "gat 3l"

/var/folders/r7/0255tsrx2dscctsn_23l9c300000gn/T/ipykernel_25148/1910911973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gat3l_errors["model"] = "gat 3l"


In [9]:
gat2l_errors["model"] = "gat 2l"

/var/folders/r7/0255tsrx2dscctsn_23l9c300000gn/T/ipykernel_25148/3368721764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gat2l_errors["model"] = "gat 2l"


In [10]:

# Merge with an indicator column
merged_df = gat3l_errors.merge(bert_errors, on='id', how='outer', indicator=True)

# Compute statistics
unique_in_gat3l = (merged_df['_merge'] == 'left_only').sum()
unique_in_bert = (merged_df['_merge'] == 'right_only').sum()
unique_in_both = (merged_df['_merge'] == 'both').sum()
total_in_gat3l = gat3l_errors['id'].nunique()
total_in_bert = bert_errors['id'].nunique()

# Print results
print("Number of unique IDs in GAT 3l errors:", unique_in_gat3l)
print("Number of unique IDs in BERT no context errors:", unique_in_bert)
print("Number of unique IDs that are errors for both models:", unique_in_both)
print("Number of errors for GAT 3l:", total_in_gat3l)
print("Number of errors for BERT no context:", total_in_bert)


Number of unique IDs in GAT 3l errors: 38
Number of unique IDs in BERT no context errors: 45
Number of unique IDs that are errors for both models: 141
Number of errors for GAT 3l: 179
Number of errors for BERT no context: 186


In [11]:
unique_in_gat3l = merged_df[(merged_df['_merge'] == 'left_only')]
unique_in_bert = merged_df[(merged_df['_merge'] == 'right_only')]


# Save rows_only_in_df1 as a CSV file
unique_in_gat3l.to_csv('errors_for_gat3l_but_not_bert.csv', index=False)

# Save rows_only_in_df2 as a CSV file
unique_in_bert.to_csv('errors_for_bert_but_not_gat3l.csv', index=False)

### Context boolean 

In [12]:
bert_errors = bert_results[(bert_results["fp"] == 1) | (bert_results["fn"] == 1)]
gat3l_errors = gat3l_results[(gat3l_results["fp"] == 1) | (gat3l_results["fn"] == 1)]
gat2l_errors = gat2l_results[(gat2l_results["fp"] == 1) | (gat2l_results["fn"] == 1)]

In [13]:
# Group by context_boolean and sum up the fp, fn, tp, and tn columns
summary_bert = bert_results.groupby('anno_ctx')[['fp', 'fn', 'tp', 'tn']].sum().reset_index()
summary_gat3l = gat3l_results.groupby('anno_ctx')[['fp', 'fn', 'tp', 'tn']].sum().reset_index()

# Display the summary tables
print("Summary for BERT:")
print(summary_bert)

print("\nSummary for GAT 3l :")
print(summary_gat3l)

Summary for BERT:
          anno_ctx  fp  fn   tp  tn
0   CurrentContent   2  51  187   0
1  PreviousContent   1  38   72   2

Summary for GAT 3l :
          anno_ctx  fp  fn   tp  tn
0   CurrentContent   2  34  204   0
1  PreviousContent   1  28   82   2


### Results and IC

In [16]:
# Third set of F1 scores
f1_scores= {
    "BERT (no context)" : np.array([0.75806, 0.75497, 0.73654, 0.75632, 0.72859]),
    "BERT concat embeddings (all conv)": np.array([0.63934, 0.53741, 0.63934, 0.63934, 0.63934]),
    "Longf concat context":  np.array([0.7668, 0.73727, 0.7358, 0.75098, 0.741]), 
    "Gat 1l": np.array([0.75034, 0.74929, 0.75033, 0.73741, 0.75493]),
    "Gat 2l": np.array([0.76533, 0.76501, 0.74834, 0.76294, 0.75683]),
    "Gat 3l": np.array([0.77411, 0.76228, 0.75275, 0.75467, 0.7655]),
    "Gat 4l": np.array([0.7644, 0.76319, 0.74716, 0.76883, 0.74967]),
}

model_names, mean_f1_scores, stds, conf_ints, ci_lowers, ci_uppers  = [], [], [], [], [], []

n = 5
t = 2.70 #2.776 # t value for n=5 and a 95% confidence interval 


for model, f1s in f1_scores.items():
    model_names.append(model)
    mean_f1 = np.mean(f1s)
    mean_f1_scores.append(mean_f1)

    # Compute standard deviation
    std_ = np.std(f1s, ddof=1)
    stds.append(std_)

    # Compute confidence interval
    conf = t * (std_ / np.sqrt(n))
    conf_ints.append(conf)
    ci_lowers.append(mean_f1 - conf)
    ci_uppers.append(mean_f1 + conf)

# Create a DataFrame
df = pd.DataFrame({
    'Model Name': model_names,
    'Mean F1': mean_f1_scores,
    'Standard Deviation': stds,
    'Confidence Interval': conf_ints,
    'Confidence Interval Lower': ci_lowers,
    'Confidence Interval Upper': ci_uppers,
})


In [17]:
df

,Model Name,Mean F1,Standard Deviation,Confidence Interval,Confidence Interval Lower,Confidence Interval Upper
0,BERT (no context),0.746896,0.013426,0.012008,0.734888,0.758904
1,BERT concat embeddings (all conv),0.618954,0.045584,0.040772,0.578182,0.659726
2,Longf concat context,0.746370,0.012865,0.011507,0.734863,0.757877
3,Gat 1l,0.748460,0.006551,0.005860,0.742600,0.754320
4,Gat 2l,0.759690,0.007205,0.006445,0.753245,0.766135
5,Gat 3l,0.761862,0.008634,0.007722,0.754140,0.769584
6,Gat 4l,0.758650,0.009617,0.008602,0.750048,0.767252
